In [3]:
# Import the libaries
import numpy as np
import numba
from numba import cuda, types, float32, int32

<br>

# Intro

---

There are several programming models for programming GPUs, including CUDA, OpenCL, and DirectCompute. In this tutorial, we will focus on CUDA using numba.

Breakdown of GPU:
- __Thread__: A single unit of computation. Think of it as a worker in a factory, performing a specific task.

- __Block__: A group of threads working together. Imagine it as a team of workers in the factory, each doing their part to complete a larger job.

- __Grid__: A grid is a collection of blocks. It is a higher level of organization for parallel computation. You can think of a grid as an entire factory with multiple teams (blocks) of workers (threads).

A __kernel__ is a function that runs on the GPU, executed by multiple threads in parallel. It is the core piece of code that you want to accelerate using the GPU.

In [5]:
##########################
#    CURRENT GPU INFO    #
##########################

# Detect the available GPUs
cuda.detect()

# Get the current GPU device
device = cuda.get_current_device()

# Print device properties
print(f"Device name: {device.name}")
print(f"Compute capability: {device.compute_capability}")
print(f"Maximum threads per block: {device.MAX_THREADS_PER_BLOCK}")
print(f"Maximum block dimensions: ({device.MAX_BLOCK_DIM_X}, {device.MAX_BLOCK_DIM_Y}, {device.MAX_BLOCK_DIM_Z})")
print(f"Maximum grid dimensions: ({device.MAX_GRID_DIM_X}, {device.MAX_GRID_DIM_Y}, {device.MAX_GRID_DIM_Z})")

Found 1 CUDA devices
id 0    b'NVIDIA GeForce GTX 1050 Ti'                              [SUPPORTED]
                      Compute Capability: 6.1
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-babb66a5-ecc1-224b-b27f-ff53d2910c66
                                Watchdog: Enabled
                            Compute Mode: WDDM
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported
Device name: b'NVIDIA GeForce GTX 1050 Ti'
Compute capability: (6, 1)
Maximum threads per block: 1024
Maximum block dimensions: (1024, 1024, 64)
Maximum grid dimensions: (2147483647, 65535, 65535)


<br>

# Device Funtion Compilation

---

Numba is a just-in-time (JIT) compiler for Python that allows users to accelerate code written in pure Python or NumPy with the help of graphics processing units (GPUs) or multicore CPUs.

__Device functions__ are functions that are only callable from within a CUDA kernel and are not callable from the host. These functions are used to perform small computations that can be parallelized on the GPU.

`@cuda.jit(device=True)` is a decorator used to define __device functions__. In other word, it can only be called from other functions running on the device (i.e. GPU).

`@cuda.jit` is a decorator used to define __kernels__. In other word, it can be called from host (i.e. CPU) to perform parallel computations on device (i.e. GPU).

In [ ]:
# Define the kernel function to multiply two arrays
@cuda.jit
def multiply_kernel(a, b, c):
    i = cuda.grid(1)                    # Get the thread ID
    if i < a.shape[0]:                  # Check if the thread is within the array bounds
        c[i] = multiply(a[i], b[i])     # Call the device function to multiply two elements

# Define the device function to multiply two elements
@cuda.jit(device=True)
def multiply(x, y):
    return x * y                        # Perform the multiplication

# Define the kernel function to add two arrays
@cuda.jit
def add_kernel(a, b, c):
    i = cuda.grid(1)                     # Get the thread ID
    if i < a.shape[0]:                   # Check if the thread is within the array bounds
        c[i] = a[i] + b[i]               # Perform the addition


<br>

# Thread, Block, and Grid Indexing

---

Compute current __thread index__:
- `cuda.threadIdx.x`: thread index along the x-axis
- `cuda.threadIdx.y`: thread index along the y-axis
- `cuda.threadIdx.z`: thread index along the z-axis

Compute __global thread index__:
- `tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x`

Compute current __block index__:
- `cuda.blockIdx.x`: block index along the x-axis
- `cuda.blockIdx.y`: block index along the y-axis
- `cuda.blockIdx.z`: block index along the z-axis

__Grid indexing__:
- `cuda.grid()`: total number of threads in the grid along each axis
- `n_threads = cuda.grid(1)`: total number of threads along the x-axis.
- `n_threads = cuda.grid(2)`: total number of threads along the y-axis
- `n_threads = cuda.grid(3)`: total number of threads along the z-axis

__Block size__ and __grid size__:
- `cuda.blockDim`: Get the block size along each axis 
- `cuda.gridDim`: Get the grid size along each axis

You cannot __print__ an statement on GPU. You must first transfer it to CPU and then print it.

In [25]:
# Sample function for printing the current thread, block, and grid dimensions
@cuda.jit
def sample_kernel(output):
    
    # Thread coordinates (currently)
    x, y, z = cuda.threadIdx.x, cuda.threadIdx.y, cuda.threadIdx.z

    # Block coordinates (currently)
    bx, by, bz = cuda.blockIdx.x, cuda.blockIdx.y, cuda.blockIdx.z

    # Grid dimensions
    grid_dim_x, grid_dim_y, grid_dim_z = cuda.gridDim.x, cuda.gridDim.y, cuda.gridDim.z

    # Index for each thread in the grid
    index = x + bx * cuda.blockDim.x + \
            (y + by * cuda.blockDim.y) * grid_dim_x * cuda.blockDim.x + \
            (z + bz * cuda.blockDim.z) * grid_dim_x * cuda.blockDim.x * grid_dim_y * cuda.blockDim.y

    # Global thread index
    tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x

    # Store the information in the output array
    output[index] = index

In [ ]:
# Define the grid and block dimensions
threads_per_block = (4, 4, 1)
blocks_per_grid = (2, 2, 1)

# Calculate the total number of threads in the grid
total_threads = threads_per_block[0] * threads_per_block[1] * threads_per_block[2] * \
                blocks_per_grid[0] * blocks_per_grid[1] * blocks_per_grid[2]

# Create an empty array to store the output
output = np.zeros(total_threads, dtype=np.int32)

# Launch the kernel on the GPU
sample_kernel[blocks_per_grid, threads_per_block](output)

In [22]:
# Copy the result from the GPU to the CPU
cuda.synchronize()

In [23]:
# Print the output
for i, index in enumerate(output):
    print(f"Index {i}: {index}")


Index 0: 0
Index 1: 1
Index 2: 2
Index 3: 3
Index 4: 4
Index 5: 5
Index 6: 6
Index 7: 7
Index 8: 8
Index 9: 9
Index 10: 10
Index 11: 11
Index 12: 12
Index 13: 13
Index 14: 14
Index 15: 15
Index 16: 16
Index 17: 17
Index 18: 18
Index 19: 19
Index 20: 20
Index 21: 21
Index 22: 22
Index 23: 23
Index 24: 24
Index 25: 25
Index 26: 26
Index 27: 27
Index 28: 28
Index 29: 29
Index 30: 30
Index 31: 31
Index 32: 32
Index 33: 33
Index 34: 34
Index 35: 35
Index 36: 36
Index 37: 37
Index 38: 38
Index 39: 39
Index 40: 40
Index 41: 41
Index 42: 42
Index 43: 43
Index 44: 44
Index 45: 45
Index 46: 46
Index 47: 47
Index 48: 48
Index 49: 49
Index 50: 50
Index 51: 51
Index 52: 52
Index 53: 53
Index 54: 54
Index 55: 55
Index 56: 56
Index 57: 57
Index 58: 58
Index 59: 59
Index 60: 60
Index 61: 61
Index 62: 62
Index 63: 63


<br>

__Set thead, block size when launching a kernel__:
- When launching a kernel, use the `threadsperblock` parameter of [number of threads per block along the x-axis, number of threads per block along the y-axis, number of threads per block along the z-axis]. For example, to launch a kernel with 64 threads per block along the x-axis, we can use:
```python
my_kernel[64, 1, 1](...)
```
- When launching a kernel, use the `blockspergrid` parameter of [(number of blocks per grid along the x-axis,), (number of blocks per grid along the y-axis,), (number of blocks per grid along the z-axis,)]. For example, to launch a kernel with 16 blocks per grid along the x-axis, we can use:
```python
my_kernel[(16,), (64,), (64,)](...)
```


In [27]:
@cuda.jit
def vector_add(a, b, c):
    i = cuda.grid(1) # get global thread index in 1D
    if i < a.shape[0]:
        c[i] = a[i] + b[i]

def main():
    n = 1024
    a = cuda.to_device(np.ones(n))
    b = cuda.to_device(np.ones(n))
    c = cuda.device_array(n)
    threads_per_block = 128
    blocks_per_grid = (n + threads_per_block - 1) // threads_per_block
    vector_add[blocks_per_grid, threads_per_block](a, b, c)
    result = c.copy_to_host()

<br>

# Memory Hierarchy

---

Breakdown of memory in GPU: There are different types of memory in a GPU, each with different access speeds and scope. The main ones are:
- __Global memory__: Accessible by all threads, but with higher latency.
- __Shared memory__: Faster memory shared within a block, enabling cooperation between threads in the same block.
- __Local memory__: Private memory for each thread.
- __Constant and texture memory__: Special types of memory with caching, used for specific purposes.

In [ ]:
# Define a CUDA kernel
@cuda.jit
def memory_demo(input_array, output_array):

    # Global memory
    idx = cuda.grid(1)
    blockDim = cuda.blockDim.x

    # Allocate shared memory array dynamically
    shared_mem = cuda.shared.array(shape=0, dtype=types.float32)

    if idx < input_array.size:
        
        # Local memory
        local_value = input_array[idx]

        # Shared memory
        shared_mem[cuda.threadIdx.x] = local_value
        cuda.syncthreads()

        # Combine the values and store in the output array
        output_array[idx] = local_value * shared_mem[cuda.threadIdx.x]

In [2]:
# Create an input and output array on the host
input_array = np.arange(16, dtype=np.float32)
output_array = np.zeros_like(input_array)

# Allocate memory on the device
d_input_array = cuda.to_device(input_array)
d_output_array = cuda.to_device(output_array)

# Define block and grid sizes
block_size = 4
grid_size = (input_array.size + block_size - 1) // block_size

# Call the CUDA kernel with dynamic shared memory size
memory_demo[grid_size, block_size, 0, block_size * 4](d_input_array, d_output_array)

# Copy the result from the GPU to the CPU
d_output_array.copy_to_host(output_array)

# Print the output array
print("Output Array:", output_array)


c:\Users\Soheil.Mohammadpour\.conda\envs\predator\lib\site-packages\numba\cuda\dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Output Array: [  0.   1.   4.   9.  16.  25.  36.  49.  64.  81. 100. 121. 144. 169.
 196. 225.]


<br>

# Numba Decorator

---

1. `@jit`: This decorator is used to mark a function for just-in-time (JIT) compilation. When this decorator is applied to a function, Numba compiles the Python code into native machine code for improved performance. 

In [ ]:
from numba import jit

@jit
def my_func(x, y):
    return x + y


2. `@njit`: This decorator is similar to @jit, but is more strict in its type checking. It requires all types to be explicitly defined and will raise an error if any types are inferred. 

In [ ]:
from numba import njit

@njit(int32(int32, int32))
def my_func(x, y):
    return x + y


3. `@vectorize`: This decorator is used to create a universal function that can operate on numpy arrays of any shape and size. It is used to parallelize element-wise computations across arrays.

In [ ]:
from numba import vectorize, int32

@vectorize([int32(int32, int32)])
def my_func(x, y):
    return x + y


4. `@guvectorize`: This decorator is similar to @vectorize, but it allows for more complex element-wise computations that may require more than two inputs. It is used to parallelize element-wise computations across arrays. 

In [ ]:
from numba import guvectorize, int32

# In this example, x and y are input arrays of size n, and res is an output array of size n. The function computes res[i] = x[i] + y[i] for each element in the arrays.
@guvectorize([(int32[:], int32[:], int32[:])], '(n), (n) -> (n)')
def my_func(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] + y[i]


<br>

# Synchronization 

---

In this section, you'll use the tools for controlling the order and timing of the operations in parallel computing.

__Synchronization__ is the coordination of multiple threads or processes to ensure that they execute in a specific order or at a specific time. In parallel computing, synchronization is important to prevent race conditions and to ensure that operations are performed correctly.

Synchronization is achieved using synchronization primitives such as barriers and events:
1. __Barriers__: A barrier is a synchronization primitive that forces threads to wait until all other threads in the group have reached the barrier before allowing any thread to proceed.  The `--syncthreads()` function is used to synchronize threads within a block. All threads in the block must reach the `--syncthreads()` call before any thread can proceed beyond it.


In [5]:
# In this kernel, each thread increments the element a[i] and then waits for all other threads in the block to complete this operation using the __syncthreads() function. After the barrier, each thread multiplies the element a[i] by 2.
@cuda.jit
def kernel(a):
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = tid + bid * bdim
    a[i] += 1
    cuda.syncthreads()
    a[i] *= 2


2. __Events__: An event is a synchronization primitive that allows threads to synchronize their execution across multiple blocks. Events are implemented using the cudaEvent_t data type and a set of functions to create, record, and wait for events.

In [6]:
import numpy as np
from numba import cuda

@cuda.jit
def kernel1(a):
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = tid + bid * bdim
    a[i] += 1

@cuda.jit
def kernel2(a):
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = tid + bid * bdim
    a[i] *= 2

a = np.ones(1024).astype(np.float32)
stream = cuda.stream()
event1 = cuda.event()
event2 = cuda.event()

kernel1[64, 16, stream](a)
event1.record(stream)
kernel2[64, 16, stream](a)
event2.record(stream)
event1.synchronize()
event2.synchronize()


c:\Users\Soheil.Mohammadpour\.conda\envs\predator\lib\site-packages\numba\cuda\dispatcher.py:488: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\Soheil.Mohammadpour\.conda\envs\predator\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
c:\Users\Soheil.Mohammadpour\.conda\envs\predator\lib\site-packages\numba\cuda\dispatcher.py:488: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\Soheil.Mohammadpour\.conda\envs\predator\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


<br>

# Memory Management 

---

__Memory management__ is a crucial aspect of programming for GPUs, as it directly affects the performance of the application, and it can be used to optimize your codes. 

Numba provides several functions for managing memory on the GPU

1. `cuda.to_device()`:  It's used to copy a NumPy array to the device memory. It takes a NumPy array as input and returns a DeviceNDArray (i.e. array on the device). The device array can be used in CUDA kernels

In [14]:
# Create a NumPy array
a = np.array([1, 2, 3])

# Copy the array to device memory
d_a = cuda.to_device(a)
d_a


2. `cuda.device_array()`:  It's used to allocate a device memory buffer of a specified size. It takes the size of the buffer in bytes as input and returns a DeviceNDArray object that represents the buffer.

In [13]:
# Allocate device memory buffer
d_a = cuda.device_array(10, dtype=np.float32)
d_a

3. `cuda.device_array_like()`: It's similar to the previous function but instead of specifying the size of the buffer, it takes a NumPy array as input and allocates a device memory buffer of the same size and data type as the input array. It returns a DeviceNDArray object that represents the allocated buffer.

In [12]:
# Create a NumPy array
a = np.array([1, 2, 3], dtype=np.float32)

# Allocate a device memory buffer of the same size and data type as a
d_a = cuda.device_array_like(a)
d_a

4. `cuda.pinned_array()`: It's used to allocate pinned host memory, which can be accessed faster by the GPU. It takes the size of the pinned memory buffer in bytes as input and returns a NumPy array that represents the pinned memory buffer.

In [11]:
# Allocate pinned host memory
pinned_a = cuda.pinned_array(10, dtype=np.float32)
pinned_a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

<br>

# Data Transfer; Host-to-device and device-to-host

---

Numba provides easy-to-use memory transfer functions for moving data between the CPU and GPU memory. 

1. Host-to-Device Data Transfer: The `cuda.to_device()` function is used to transfer data from the host (CPU) to the device (GPU) memory.

In [15]:
# Create a NumPy array
a = np.array([1, 2, 3, 4])

# Transfer the array to the GPU memory
d_a = cuda.to_device(a)
d_a

2. Device-to-Host Data Transfer: The `copy_to_host()` function is used to transfer data from the device (GPU) memory to the host (CPU) memory.

In [16]:
# Create a device array
d_a = cuda.device_array(4)

# Transfer the array to the CPU memory
a = d_a.copy_to_host()
a

array([0., 0., 0., 0.])

<br>

# 

---


Here is a list of topics you should learn to get started with Numba and CUDA:

Python and NumPy: Familiarize yourself with Python and NumPy as they are the foundation for using Numba.

Numba basics:

Just-In-Time (JIT) compilation
Numba decorators: @jit, @njit, @vectorize, @guvectorize
Supported Python and NumPy features in Numba
CUDA basics:

GPU architecture and programming model
CUDA memory hierarchy: global, shared, local, and constant memory
CUDA threads, thread blocks, and grids
Synchronization primitives: barriers and events
Numba for CUDA:

Numba CUDA decorators: @cuda.jit, @cuda.vectorize, @cuda.reduce
Memory management with Numba: cuda.to_device(), cuda.device_array(), cuda.device_array_like(), cuda.pinned_array()
Host-to-device and device-to-host data transfers: copy_to_device(), copy_to_host()
Device function compilation with @cuda.jit(device=True)
Thread, block, and grid indexing: cuda.grid(), cuda.gridsize(), cuda.threadIdx, cuda.blockIdx, cuda.blockDim, cuda.gridDim
Synchronization: cuda.syncthreads()
Atomic operations: cuda.atomic.add(), cuda.atomic.max(), etc.
Dynamic shared memory allocation
Performance optimization:

Coalesced memory access patterns
Shared memory utilization
Occupancy maximization
Thread divergence and warp execution
Profiling tools: Nsight Compute, Nsight Systems, and nvprof
Performance optimization strategies: loop unrolling, constant memory utilization, etc.
Interoperability:

Interfacing Numba with other GPU libraries like CuPy or PyTorch
Interfacing Numba with C/C++ code using the Python C-API, ctypes, or CFFI